In [3]:
# Plot error space for one example of case A.2

# Import libraries
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.tri as tri
import sys
sys.path.insert(1, '../../src')

# Import functions
from EM1D import EMf_2Lay_HVP, GlobalSearch_2Lay

In [7]:
# Load model and data
model_A2_1 = np.load('models/model_synth_2Lay_A2_1.npy')
model_A2_2 = np.load('models/model_synth_2Lay_A2_2.npy')
model_A2_3 = np.load('models/model_synth_2Lay_A2_3.npy')
model_A2_4 = np.load('models/model_synth_2Lay_A2_4.npy')

data_A2_1 = np.load('data/data_synth_2Lay_A2_1.npy')
data_A2_2 = np.load('data/data_synth_2Lay_A2_2.npy')
data_A2_3 = np.load('data/data_synth_2Lay_A2_3.npy')
data_A2_4 = np.load('data/data_synth_2Lay_A2_4.npy')

LUT = np.load('../data/LUTable_2Lay.npy')
conds = np.load('../data/conds.npy')
thicks = np.load('../data/thicks.npy')

# Load survey parameters
survey = np.load('../data/survey_2Lay.npy', allow_pickle=True).item()
offsets = survey['offsets']
height = survey['height']
freq = survey['freq']
lambd = survey['lambd']
filt = survey['filt']

# Load estimated model from Global Search
model_GS_A2_1 = np.load('results/model_2Lay_A2_1.npy')
model_GS_A2_2 = np.load('results/model_2Lay_A2_2.npy')
model_GS_A2_3 = np.load('results/model_2Lay_A2_3.npy')
model_GS_A2_4 = np.load('results/model_2Lay_A2_4.npy')

# Load estimated models from Optimization
model_Opt_A2_1 = np.load('results/model_Opt_A2_1.npy')
model_Opt_A2_2 = np.load('results/model_Opt_A2_2.npy')
model_Opt_A2_3 = np.load('results/model_Opt_A2_3.npy')
model_Opt_A2_4 = np.load('results/model_Opt_A2_4.npy')

# For example let's check the 1D model following position
pos = 10
nsl = 121

In [11]:
def NoiseAnalysis_GS_2Lay(data_true, noise=0.1, test_it=100):
    
    models_GS = []
    for i in range(test_it):
        data_noise = data_true* (1 +np.random.normal(size=len(data_true))*noise)
        models_GS.append(GlobalSearch_2Lay(LUT, data_noise, conds, thicks, nsl))
    return models_GS 

def NoiseAnalysis_Opt_2Lay(data_true, noise=0.1, test_it=100):
    m0 = [3, 500/1000, 500/1000]
    lam = 0
    models_Opt = []
    
    for i in range(test_it):
        EMf = EMf_2Lay_Opt_HVP_1D(lambd, height, offsets, freq, filt, nlay=2)

        transThk = pg.trans.TransLogLU(0.1,7)
        transSig = pg.trans.TransLogLU(10/1000,2000/1000)
        # Define transformation
        EMf.region(0).setTransModel(transThk)
        EMf.region(1).setTransModel(transSig)

        # Define inversion framework from pygimli
        invEM = pg.Inversion()
        invEM.setForwardOperator(EMf) # set forward operator

        # Relative error array
        error = 1e-3 # relative error
        relativeError = np.ones_like(data_true[0]) * error
        data_noise = data_true* (1 +np.random.normal(size=len(data_true))*noise)
        models_Opt.append(invEM.run(data_noise, relativeError, startModel= m0, lam=lam, verbose=False))
    return models_Opt


In [ ]:
models_noise_GS_A2_1 = NoiseAnalysis_GS_2Lay(data_A2_1[pos])
models_noise_GS_A2_2 = NoiseAnalysis_GS_2Lay(data_A2_2[pos])
models_noise_GS_A2_3 = NoiseAnalysis_GS_2Lay(data_A2_3[pos])
models_noise_GS_A2_4 = NoiseAnalysis_GS_2Lay(data_A2_4[pos])

In [ ]:
models_noise_Opt_A2_1 = NoiseAnalysis_Opt_2Lay(data_A2_1[pos])
models_noise_Opt_A2_2 = NoiseAnalysis_Opt_2Lay(data_A2_2[pos])
models_noise_Opt_A2_3 = NoiseAnalysis_Opt_2Lay(data_A2_3[pos])
models_noise_Opt_A2_4 = NoiseAnalysis_Opt_2Lay(data_A2_4[pos])

In [ ]:
print('hello')